# Creating the Dataset

This notebook creates, from the c3d complete data, all the files required for both the regression and the Cables visualisation later, keeping only the points that are of interest for our project.

In [2]:
import c3d
import pandas as pd
import numpy as np
import os
import json

In [34]:
# Handy function to read a c3d file to a DataFrame
def read_c3d_file(filename):

    reader = c3d.Reader(open(filename, 'rb'))

    rows_list = []
    for i, points, analog in reader.read_frames():
            dict1 = {}
            # get input row in dictionary format
            dict1.update({"Frame" : i, "Points" : points}) 
            rows_list.append(dict1)
            
    return pd.DataFrame(rows_list)    

In [130]:
# Gets the coordinates we are interested in from the list of all cooridnates
def get_coordinates(filename, points):
    df = read_c3d_file(filename)
    
    coordinates = np.empty((1,3,len(df.index)))
    
    for point in points:
        points_motion = pd.DataFrame(df['Points']).stack().str[point].unstack()['Points']
        X = pd.DataFrame(points_motion).stack().str[0].unstack()['Points']
        Y = pd.DataFrame(points_motion).stack().str[1].unstack()['Points']
        Z = pd.DataFrame(points_motion).stack().str[2].unstack()['Points']
        XYZ = np.array([X,Y,Z])
        coordinates = np.concatenate([coordinates, [XYZ]], axis=0)
    np.save(os.path.splitext(filename)[0] +'.npy', coordinates)
    return coordinates

In [133]:
# Reads through all the directory to process all the files
for filename in os.listdir('./Data/c3d'):
    if filename.endswith(".c3d"):
        get_coordinates('./Data/c3d/'+filename, [2, 8, 17, 26])[1:]  # 2,8,17 and 26 correspond respectively to the 
                                                                     # right foot, left foot, right hand, left hand
        continue
    else:
        continue

In [11]:
# To convert from numpy array to json, useful for Cables
def change_format(file):
    arr = np.load(file)
    l = arr.tolist()
    with open('./Data/c3d/'+ os.path.splitext(filename)[0] + '.json', 'w') as f:
        json.dump(l, f)

In [12]:
# Change format for Cables
for filename in os.listdir('./Data/c3d'):
    if filename.endswith(".npy"):
        change_format('./Data/c3d/'+filename)
        continue
    else:
        continue